In [ ]:
!pip install snorkel

In [102]:
import pandas as pd

<IPython.core.display.Javascript object>

In [164]:
train_df = pd.read_pickle("train_fake_jobs.pkl")
test_df = pd.read_pickle("test_fake_jobs.pkl")

<IPython.core.display.Javascript object>

In [226]:
from dtale import show

show(train_df)

<IPython.core.display.Javascript object>

Exception occurred while processing request: argument of type 'int' is not iterable
Traceback (most recent call last):
  File "/home/khuyen/Data-science/venv/lib/python3.8/site-packages/pandas/core/computation/ops.py", line 271, in _in
    return x.isin(y)
AttributeError: 'list' object has no attribute 'isin'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/khuyen/Data-science/venv/lib/python3.8/site-packages/dtale/views.py", line 114, in _handle_exceptions
    return func(*args, **kwargs)
  File "/home/khuyen/Data-science/venv/lib/python3.8/site-packages/dtale/views.py", line 1787, in test_filter
    run_query(
  File "/home/khuyen/Data-science/venv/lib/python3.8/site-packages/dtale/query.py", line 69, in run_query
    df = df.query(
  File "/home/khuyen/Data-science/venv/lib/python3.8/site-packages/pandas/core/frame.py", line 4060, in query
    res = self.eval(expr, **kwargs)
  File "/home/khuyen/Data-science/venv/

2022-01-27 15:11:23,655 - ERROR    - Exception occurred while processing request: argument of type 'int' is not iterable
Traceback (most recent call last):
  File "/home/khuyen/Data-science/venv/lib/python3.8/site-packages/pandas/core/computation/ops.py", line 271, in _in
    return x.isin(y)
AttributeError: 'list' object has no attribute 'isin'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/khuyen/Data-science/venv/lib/python3.8/site-packages/dtale/views.py", line 114, in _handle_exceptions
    return func(*args, **kwargs)
  File "/home/khuyen/Data-science/venv/lib/python3.8/site-packages/dtale/views.py", line 1787, in test_filter
    run_query(
  File "/home/khuyen/Data-science/venv/lib/python3.8/site-packages/dtale/query.py", line 69, in run_query
    df = df.query(
  File "/home/khuyen/Data-science/venv/lib/python3.8/site-packages/pandas/core/frame.py", line 4060, in query
    res = self.eval(expr, **kwargs)
  

In [104]:
from snorkel.labeling import labeling_function
from snorkel.labeling import PandasLFApplier
from snorkel.labeling import LFAnalysis
from snorkel.preprocess.nlp import SpacyPreprocessor


<IPython.core.display.Javascript object>

In [105]:
FAKE = 1
REAL = 0
ABSTAIN = -1

<IPython.core.display.Javascript object>

In [263]:
@labeling_function()
def no_requirements(x: pd.Series):

    return FAKE if x.requirements == "" else ABSTAIN


@labeling_function()
def requiments_less_than_10(x: pd.Series):
    num_words = len(x.requirements.split(" "))
    return FAKE if num_words <= 10 else ABSTAIN


@labeling_function()
def requiments_less_than_20(x: pd.Series):
    num_words = len(x.requirements.split(" "))
    return FAKE if num_words <= 20 else ABSTAIN


@labeling_function()
def no_company_profile(x: pd.Series):
    return FAKE if x.company_profile == "" else ABSTAIN


@labeling_function()
def suspicious_company(x: pd.Series):
    fake_companies = [
        "Aker Solutions",
        "Aptitude Staffing Solutions",
        "Gary Cartwright",
        "Edison International and Refined Resources",
        "Le Meridien",
    ]
    return (
        FAKE
        if any(
            company.lower() in x.company_profile.lower() for company in fake_companies
        )
        else ABSTAIN
    )


@labeling_function()
def no_company_logo(x: pd.Series):
    return FAKE if x.has_company_logo == 0 else ABSTAIN


@labeling_function()
def no_profile_and_suspicious_company(x: pd.Series):
    fake_companies = [
        "Aker Solutions",
        "Aptitude Staffing Solutions",
        "Gary Cartwright",
        "Edison International and Refined Resources",
        "Le Meridien",
    ]
    return (
        FAKE
        if any(
            company.lower() in x.company_profile.lower() for company in fake_companies
        )
        or x.company_profile == ""
        or x.has_company_logo == 0
        else ABSTAIN
    )


@labeling_function()
def has_background_check(x: pd.Series):
    return REAL if "background check" in x.requirements else ABSTAIN


@labeling_function()
def required_experience(x: pd.Series):
    return REAL if x.required_experience else ABSTAIN


@labeling_function()
def required_education(x: pd.Series):
    return REAL if x.required_education else ABSTAIN

<IPython.core.display.Javascript object>

In [89]:
%%ipytest -qq

test = [('', FAKE),]

@pytest.mark.parametrize("test_input, expected", test)
def test_no_company_profile(test_input, expected):
    input_df = pd.DataFrame({'company_profile': [test_input]})
    assert no_company_profile(input_df.iloc[0]) == expected

<IPython.core.display.Javascript object>

.                                                                                            [100%]


<IPython.core.display.Javascript object>

In [90]:
%%ipytest -qq

test = [("hello abc  ", FAKE),
        ("", FAKE)]

@pytest.mark.parametrize("test_input, expected", test)
def requiments_less_than_10(test_input, expected):
    input_df = pd.DataFrame({'requirements': [test_input]})
    assert requiments_less_than_10(input_df.iloc[0]) == expected

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [156]:
%%ipytest -qq

test = [("Aker Solutions is a global provider of products", FAKE)]

@pytest.mark.parametrize("test_input, expected", test)
def test_suspicious_company(test_input, expected):
    input_df = pd.DataFrame({'company_profile': [test_input]})
    assert suspicious_company(input_df.iloc[0]) == expected

<IPython.core.display.Javascript object>

.                                                                                            [100%]


<IPython.core.display.Javascript object>

In [39]:
%%ipytest -qq

test = [(0, FAKE), (1, ABSTAIN)]

@pytest.mark.parametrize("test_input, expected", test)
def no_company_logo(test_input, expected):
    input_df = pd.DataFrame({'has_company_logo': [test_input]})
    assert no_company_logo(input_df.iloc[0]) == expected

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [129]:
lfs = [
    requiments_less_than_10,
    requiments_less_than_20,
    no_requirements,
    no_company_profile,
    suspicious_company,
    no_company_logo,
    has_background_check,
]

applier = PandasLFApplier(lfs=lfs)
L_train = applier.apply(df=train_df)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 13410/13410 [00:01<00:00, 13292.66it/s]


<IPython.core.display.Javascript object>

In [109]:
LFAnalysis(L=L_train, lfs=lfs).lf_summary(train_df.fraudulent.values)

/home/khuyen/Data-science/venv/lib/python3.8/site-packages/sklearn/utils/validation.py:70: FutureWarning: Pass labels=[-1  0  1] as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  warnings.warn(f"Pass {args_msg} as keyword args. From version "
/home/khuyen/Data-science/venv/lib/python3.8/site-packages/sklearn/utils/validation.py:70: FutureWarning: Pass labels=[-1  0  1] as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  warnings.warn(f"Pass {args_msg} as keyword args. From version "
/home/khuyen/Data-science/venv/lib/python3.8/site-packages/sklearn/utils/validation.py:70: FutureWarning: Pass labels=[-1  0  1] as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  warnings.warn(f"Pass {args_msg} as keyword args. From version "
/home/khuyen/Data-science/venv/lib/python3.8/site-packages/sklearn/uti

,j,Polarity,Coverage,Overlaps,Conflicts,Correct,Incorrect,Emp. Acc.
requiments_less_than_10,0,[1],0.173304,0.173304,0.0,164,2160,0.070568
requiments_less_than_20,1,[1],0.211186,0.184042,0.0,213,2619,0.075212
no_requirements,2,[1],0.150708,0.150708,0.0,115,1906,0.056903
no_company_profile,3,[1],0.186204,0.155481,0.0,459,2038,0.183821
suspicious_company,4,[],0.000000,0.000000,0.0,0,0,0.000000
no_company_logo,5,[1],0.205742,0.181432,0.0,459,2300,0.166365
has_phone,6,[1],0.001715,0.000447,0.0,0,23,0.000000


<IPython.core.display.Javascript object>

In [264]:
lfs = [
    #     requiments_less_than_20,
    no_company_profile,
    suspicious_company,
    no_company_logo,
    has_background_check,
    required_experience,
    required_education,
]

applier = PandasLFApplier(lfs=lfs)
L_train = applier.apply(df=train_df)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 13410/13410 [00:00<00:00, 16221.05it/s]


<IPython.core.display.Javascript object>

In [265]:
LFAnalysis(L=L_train, lfs=lfs).lf_summary(train_df.fraudulent.values)

/home/khuyen/Data-science/venv/lib/python3.8/site-packages/sklearn/utils/validation.py:70: FutureWarning:

Pass labels=[-1  0  1] as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error

/home/khuyen/Data-science/venv/lib/python3.8/site-packages/sklearn/utils/validation.py:70: FutureWarning:

Pass labels=[-1  0  1] as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error

/home/khuyen/Data-science/venv/lib/python3.8/site-packages/sklearn/utils/validation.py:70: FutureWarning:

Pass labels=[-1  0  1] as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error

/home/khuyen/Data-science/venv/lib/python3.8/site-packages/sklearn/utils/validation.py:70: FutureWarning:

Pass labels=[-1  0  1] as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error

/home/khuyen/Dat

,j,Polarity,Coverage,Overlaps,Conflicts,Correct,Incorrect,Emp. Acc.
no_company_profile,0,[1],0.186204,0.186204,0.186204,459,2038,0.183821
suspicious_company,1,[1],0.006861,0.006861,0.006861,92,0,1.000000
no_company_logo,2,[1],0.205742,0.205742,0.205742,459,2300,0.166365
has_background_check,3,[0],0.035496,0.035496,0.002759,464,12,0.974790
required_experience,4,[0],1.000000,1.000000,0.251156,12741,669,0.950112
required_education,5,[0],1.000000,1.000000,0.251156,12741,669,0.950112


<IPython.core.display.Javascript object>

In [266]:
from snorkel.labeling.model import MajorityLabelVoter

majority_model = MajorityLabelVoter()
preds_train = majority_model.predict(L=L_train)

<IPython.core.display.Javascript object>

In [267]:
from snorkel.labeling.model import LabelModel

label_model = LabelModel(cardinality=2, verbose=True)
label_model.fit(L_train=L_train, n_epochs=500, log_freq=100, seed=123)


2022-01-27 15:25:59,948 - INFO     - Computing O...
2022-01-27 15:25:59,951 - INFO     - Estimating \mu...
  0%|                                                                                                                               | 0/500 [00:00<?, ?epoch/s]2022-01-27 15:25:59,954 - INFO     - [0 epochs]: TRAIN:[loss=2.362]
2022-01-27 15:26:00,017 - INFO     - [100 epochs]: TRAIN:[loss=0.020]
 32%|█████████████████████████████████████▎                                                                             | 162/500 [00:00<00:00, 1611.34epoch/s]2022-01-27 15:26:00,077 - INFO     - [200 epochs]: TRAIN:[loss=0.006]
2022-01-27 15:26:00,123 - INFO     - [300 epochs]: TRAIN:[loss=0.002]
100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 500/500 [00:00<00:00, 1907.56epoch/s]
2022-01-27 15:26:00,216 - INFO     - Finished Training


<IPython.core.display.Javascript object>

In [268]:
L_test = applier.apply(df=test_df)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4470/4470 [00:00<00:00, 11621.94it/s]


<IPython.core.display.Javascript object>

In [269]:
Y_train = train_df["fraudulent"]
Y_test = test_df["fraudulent"]

<IPython.core.display.Javascript object>

In [270]:
majority_acc = majority_model.score(L=L_test, Y=Y_test, tie_break_policy="random")[
    "accuracy"
]
print(f"{'Majority Vote Accuracy:':<25} {majority_acc * 100:.1f}%")

Majority Vote Accuracy:   91.1%


<IPython.core.display.Javascript object>

In [271]:
label_model_acc = label_model.score(L=L_test, Y=Y_test, tie_break_policy="random")[
    "accuracy"
]
print(f"{'Label Model Accuracy:':<25} {label_model_acc * 100:.1f}%")

Label Model Accuracy:     77.4%


<IPython.core.display.Javascript object>

In [291]:
predictions = majority_model.predict(L_test)

<IPython.core.display.Javascript object>